# Learning Implementation for Stochastic Linear Regression
with regularization.

In [1]:
import numpy as np
from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Observations:
* Reacts better to large bias than BatchGD
* If features have noncompareable sizes then bigger feature gets more weight
* Runs 1 step after last print  


# Query:
* Should bias update be multiplied by lr
* What diff does dividing by m makes

## Implementation

In [3]:
def added_cost(Lambda,W,m):
    """Returns added cost of regularization.
    """
    
    return (np.sum(W**2)) * Lambda/(2*m)

In [4]:
# Check for alpha and its multiplication with the reg term
def single_update(i,X,y,y_,W,b,alpha,Lambda):
    """Returns W,b after single update of ith data point.
    """
    m,n = X.shape
    res = y[i]-y_[i]
        
    dJ_dW = np.dot(res,X[i])  - Lambda*W
    dJ_db = res.mean()
    
    W += dJ_dW*alpha/m
    b += (y[i]-y_[i])*alpha

    return W,b

In [5]:
def SGD_LinearRegression(X,y, iterations = 100,alpha = 0.000001,Lambda=0.0001, output_limit=10):
    """Returns W,b after training lr using stochastic gradient descent with regularization applied.
    
    Parameter
    ---------
    iterations: int, default=100
        Number of complete iterations through X
        
    alpha: float, default=0.000001
        Constant Learning Rate
    
    Lambda: float, default=0.0001
        Rate for l2 Regularization
        
    output_limit: int, default=10
        Number of iterations to show
        
    Returns
    -------
    W: numpy.ndarray
        The optimized weights.
    b: numpy.longdouble
        The optimized itercept.
    """
    
    if output_limit<=0:
        print("Choose natural output limit!")
        return None, None
    m,n = X.shape
    W = np.zeros(n)
    b = np.float128(0)
    
    try:
        for k in range(iterations+1):
            y_ = np.dot(X,W) +b
            
            if k % (iterations//output_limit) == 0:
                print(f"({k//(iterations//output_limit)}/{output_limit}) > Iteration: {k}",
                      f"Cost: {mean_squared_error(y,y_) + added_cost(Lambda,W,m) }",
                      f"   Weights: {W}",
                      f"Bias: {b:.4f}"
                     )
                
            for i in range(m):
                W,b = single_update(i,X,y,y_,W,b,alpha,Lambda)
                y_ = np.dot(X,W) + b
    except KeyboardInterrupt:
        print(f"\nTerminated! Returned: Weights: {W}, Bias: {b}")
        return W,b
    return W,b

## Running Example

In [6]:
X = np.random.rand(50,2)
y = 5*X[:,0] + 11*X[:,1] + 5

In [7]:
with np.printoptions(precision=4):
    W,b = SGD_LinearRegression(X,y, 1000,0.25,output_limit=20)

(0/20) > Iteration: 0 Cost: 180.99629390843526    Weights: [0. 0.] Bias: 0.0000
(1/20) > Iteration: 50 Cost: 1.668062662339468    Weights: [2.7897 6.6294] Bias: 8.0433
(2/20) > Iteration: 100 Cost: 0.2887750847738598    Weights: [3.996  9.2303] Bias: 6.2812
(3/20) > Iteration: 150 Cost: 0.051780222832462725    Weights: [ 4.5397 10.2746] Bias: 5.5471
(4/20) > Iteration: 200 Cost: 0.009862095393415517    Weights: [ 4.7859 10.6957] Bias: 5.2389
(5/20) > Iteration: 250 Cost: 0.002134976427227674    Weights: [ 4.8978 10.8665] Bias: 5.1085
(6/20) > Iteration: 300 Cost: 0.0006167889210216816    Weights: [ 4.9489 10.9361] Bias: 5.0529
(7/20) > Iteration: 350 Cost: 0.00028707939503660286    Weights: [ 4.9723 10.9647] Bias: 5.0290
(8/20) > Iteration: 400 Cost: 0.00020437506824504427    Weights: [ 4.9831 10.9765] Bias: 5.0186
(9/20) > Iteration: 450 Cost: 0.0001797790305020221    Weights: [ 4.988  10.9814] Bias: 5.0141
(10/20) > Iteration: 500 Cost: 0.000171235566856874    Weights: [ 4.9903 10.98

In [8]:
# Using sklearn SGDRegressor
m = make_pipeline(
    SGDRegressor(
    max_iter=1000,
    n_iter_no_change=250, 
    penalty='l2',
    learning_rate='constant', eta0=0.01, 
    tol=0.00001
    )
)
    
m.fit(X,y)

Pipeline(steps=[('sgdregressor',
                 SGDRegressor(learning_rate='constant', n_iter_no_change=250,
                              tol=1e-05))])

In [9]:
m[0].coef_

array([ 4.99126965, 10.98366461])